## makemore: becoming a backprop ninja

In [1]:
# there no change change in the first several cells from last lecture

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# download the names.txt file from github
# !curl https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

In [4]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# ok biolerplate done, now we get to the action:

In [8]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [10]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5 
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3492, grad_fn=<NegBackward0>)

### figuring out backprop/chain rule for matrix mult

In [12]:
# a b       e f         ae+bg   af+bh
# c d   x   g h     =   ce+dg   cf+dh

# so d/da           =   e       f
#                       0       0

# d/da
# e f
# 0 0

# d/db
# g h
# 0 0

# d/dc
# 0 0
# e f

# d/dd
# 0 0
# g h 


# how does this transfer to the basic multiplication done in micrograd? 

In [13]:
a = torch.tensor([
    [3., 2., ],
    [1., 2., ],
    [1., 3., ]
])

batch = torch.tensor([
    [1., 1., 3],
    [1., 2., -1],
])

a.requires_grad = True
batch.requires_grad = True

c = a @ batch
c.retain_grad()

loss = c.exp().sum()
loss.retain_grad()

loss.backward()
loss

tensor(3665.1279, grad_fn=<SumBackward0>)

In [14]:
dc = c.exp()

In [15]:
da = dc @ batch.T
da

tensor([[4534.9458, 1245.0463],
        [ 176.6535,  314.1936],
        [1154.2313, 2246.8645]], grad_fn=<MmBackward0>)

In [16]:
db = a.T @ dc
db

tensor([[ 519.9232, 4534.9458, 3293.6177],
        [ 500.7919, 5779.9922, 2201.7029]], grad_fn=<MmBackward0>)

In [17]:
h.grad.shape, h.grad[0], h.grad[-1]

(torch.Size([32, 64]),
 tensor([-1.2897e-03, -4.8919e-03,  2.2109e-03,  1.2488e-03,  3.9155e-03,
         -1.3501e-03,  2.7849e-03,  5.3353e-05,  1.7108e-04,  4.8461e-03,
         -2.5354e-03,  8.6588e-05,  4.0724e-03,  2.4050e-03,  7.2361e-04,
         -3.4238e-03,  9.2246e-04,  3.4579e-03,  3.8256e-04,  3.7514e-03,
          4.5813e-03,  3.1364e-03,  5.1987e-03,  3.7057e-03,  2.1755e-03,
          5.2814e-03, -1.1706e-03, -6.4942e-03,  3.6987e-03,  5.6909e-03,
          1.0613e-03, -8.5855e-03,  3.2009e-04,  2.8792e-03,  2.9219e-03,
         -2.5826e-03,  2.0571e-03,  7.1230e-03, -3.5925e-03,  2.0751e-03,
         -4.4746e-04,  1.9928e-03, -4.7494e-03, -7.3342e-03, -3.3889e-03,
          3.3220e-04, -6.1904e-03, -2.1342e-03,  3.0683e-03,  2.8265e-03,
         -3.3122e-03, -3.6050e-03, -1.3301e-04,  2.8299e-03,  1.7095e-03,
         -1.4525e-03, -3.9045e-03,  4.8770e-03,  2.7876e-03,  4.4508e-03,
         -3.8753e-04, -2.0180e-03, -2.0656e-03, -1.9275e-03]),
 tensor([ 3.5875e-04,  5.2

In [18]:
logits.shape, h.shape, W2.shape, b2.shape

(torch.Size([32, 27]),
 torch.Size([32, 64]),
 torch.Size([64, 27]),
 torch.Size([27]))

## exercise

In [19]:
b2.shape
# gonna be broadcast 32 times down
# so one change gonna have a 32 times effect (?)? 

torch.Size([27])

In [20]:
C.shape, Xb.shape, C[Xb].shape, emb.shape

(torch.Size([27, 10]),
 torch.Size([32, 3]),
 torch.Size([32, 3, 10]),
 torch.Size([32, 3, 10]))

In [21]:
bndiff.shape, bndiff2.shape, bnvar_inv.shape, emb.shape

(torch.Size([32, 64]),
 torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 3, 10]))

In [22]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

dprobs = 1.0/probs * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True) #hm... this will require some thinking
dcounts = counts_sum_inv * dprobs #internal broadcasting done by pytorch

dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv

dcounts += torch.ones_like(counts) * dcounts_sum #broadcast sum to all elements

dnorm_logits = counts * dcounts #just reuse counts for e^x local derivative

dlogits = 1 * dnorm_logits.clone()
dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdim=True) #the sum makes sense, because the max is replicated and subtracted from all individual logits

# could also be done via one-hot
temp = torch.zeros_like(logits) 
temp[range(len(logits)), logits.max(1).indices] = 1

dlogits += temp * dlogit_maxes

# little bit freaky, but ok...
dh = dlogits @ W2.T
dW2 = h.T @ dlogits

db2 = dlogits.sum(0, keepdim=True) # first broadcast to [32, 27], local derivates is just ones, then with global derivative it's like taking a sum

dhpreact = (1 - h**2) * dh

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnbias = dhpreact.sum(0, keepdim=True)
dbnraw = (bngain * dhpreact) #no need to sum here because bnraw is [32, 64]

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

# 1/sqrt(x+1e-5) = (x+1e-5)**-1/2
# d/dx = -1/2(x+1e-5)**-3/2
dbnvar = -1/2*(bnvar+1e-5)**(-3/2) * dbnvar_inv

dbndiff2 = torch.ones_like(bndiff2) * 1/(n-1) * dbnvar

dbndiff = 2 * bndiff * dbndiff2 
dbndiff += bnvar_inv * dbnraw #this is the 2nd branch

dbnmeani = -dbndiff.sum(0, keepdim=True)

# two branches
dhprebn = 1.0 * dbndiff 
dhprebn += torch.ones_like(hprebn) * (1/n) * dbnmeani 

dembcat = dhprebn @ W1.T

dW1 = embcat.T @ dhprebn

db1 = 1.0 * dhprebn.sum(0, keepdim=True)

demb = 1.0 * dembcat.view(emb.shape) #unview back

# C:             [27, 10]
# Xb:            [32, 3]
# emb = C[Xb]:   [32, 3, 10]
dC = torch.zeros_like(C)

for i, batch in enumerate(Xb):
    for j, chari in enumerate(batch): 
        dC[chari] += 1.0 * demb[i, j] #probs not the best way...

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

### f*cking around

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.34920334815979 diff: -3661.77880859375


In [ ]:
# the ground truth
logits.grad[0]

tensor([ 0.0024,  0.0025,  0.0006,  0.0014,  0.0007,  0.0026,  0.0008,  0.0012,
        -0.0307,  0.0011,  0.0011,  0.0012,  0.0011,  0.0009,  0.0012,  0.0004,
         0.0003,  0.0006,  0.0005,  0.0016,  0.0016,  0.0006,  0.0008,  0.0023,
         0.0019,  0.0008,  0.0007])

In [ ]:
clogits[0]

tensor([ 0.8583,  0.9010, -0.5737,  0.3544, -0.3536,  0.9615, -0.2579,  0.1739,
        -0.5386,  0.0930,  0.0885,  0.1411,  0.0565, -0.1194,  0.1551, -0.8243,
        -1.2296, -0.5238, -0.7397,  0.4995,  0.4836, -0.4846, -0.2660,  0.8370,
         0.6354, -0.2356, -0.3809], grad_fn=<SelectBackward0>)

In [ ]:
def softmax(x):
    return x.exp() / x.exp().sum(1, keepdim=True)

In [ ]:
clogits = logits.detach().clone() 
clogits.requires_grad=True
clogits.retain_grad()

softmaxxed = softmax(clogits)
softmaxxed.retain_grad()

test = -softmaxxed.log()[range(n), Yb].mean() #key insight: this creates a split between grads of selected and unselected values
test.backward()

In [123]:
h_ = 0.001
softmaxxed_grad = (softmax(clogits + h_) - softmax(clogits)) / h_
softmaxxed_grad[0]

tensor([ 0.0000e+00,  0.0000e+00, -1.8626e-06, -3.7253e-06, -3.7253e-06,
        -7.4506e-06, -1.8626e-06,  0.0000e+00,  0.0000e+00, -3.7253e-06,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -1.8626e-06,  0.0000e+00,
        -9.3132e-07,  0.0000e+00, -1.8626e-06, -9.3132e-07, -7.4506e-06,
        -7.4506e-06, -1.8626e-06, -3.7253e-06, -7.4506e-06, -7.4506e-06,
        -3.7253e-06,  0.0000e+00], grad_fn=<SelectBackward0>)

In [ ]:
# dsoftmaxxed / dlogits
# s = e^x / sum{e^a + e^b + ... + e^x}
# ds/dx = e^x * (sum{e^a + e^b + ... + e^x} ** -1)
# = e^x * (-S**-2 * e^x) + e^x * sum(e^X) ** -1

In [59]:
maxs = logits.max(1, keepdim=True)
l = logits - maxs.values

In [60]:
# one liner
test_loss = -(l - l.exp().sum(1, keepdim=True).log())[range(n), Yb].mean()
# need to d/dl

test_loss, loss_fast, loss

(tensor(3.4770, grad_fn=<NegBackward0>),
 tensor(3.4770, grad_fn=<NllLossBackward0>),
 tensor(3.4770, grad_fn=<NegBackward0>))

In [61]:
test_loss = -(l - l.exp().sum(1, keepdim=True).log())[range(n), Yb].mean()
test_loss.shape
test_loss

tensor(3.4770, grad_fn=<NegBackward0>)

In [63]:
h = 0.001

lh = l.clone()
lh[0][0] += h

test_lossh = -(lh - lh.exp().sum(1, keepdim=True).log())[range(n), Yb].mean() 
test_lossh, ((test_lossh - test_loss)  / h)

(tensor(3.4770, grad_fn=<NegBackward0>),
 tensor(-0.0310, grad_fn=<DivBackward0>))

In [70]:
e = l.exp()
sum = e.sum(1, keepdim=True)

In [65]:
dlogits = -(1 - e / sum)
dlogits[range(n), Yb] #this is how we will impact the individual loss / logit value

tensor([-0.9816, -0.9757, -0.9891, -0.9449, -0.9733, -0.9755, -0.9661, -0.9706,
        -0.9891, -0.9723, -0.9859, -0.9520, -0.9693, -0.9680, -0.9380, -0.9708,
        -0.9533, -0.9643, -0.9671, -0.9774, -0.9191, -0.9681, -0.9368, -0.9765,
        -0.9632, -0.9129, -0.9445, -0.9815, -0.9553, -0.9693, -0.9808, -0.9818],
       grad_fn=<IndexBackward0>)

In [69]:
h = 0.001

lh = l.clone()
lh[0][0] += h

test_lossh = -(lh - lh.exp().sum(1, keepdim=True).log())[range(n), Yb]
test_lossh, ((test_lossh - test_loss)  / h)

(tensor([3.9930, 3.7168, 4.5197, 2.8979, 3.6246, 3.7073, 3.3838, 3.5261, 4.5197,
         3.5846, 4.2604, 3.0375, 3.4829, 3.4430, 2.7804, 3.5341, 3.0643, 3.3314,
         3.4149, 3.7898, 2.5149, 3.4461, 2.7618, 3.7526, 3.3013, 2.4412, 2.8908,
         3.9913, 3.1088, 3.4825, 3.9520, 4.0068], grad_fn=<NegBackward0>),
 tensor([  516.0029,   239.8224,  1042.7302,  -579.1063,   147.5861,   230.3357,
           -93.2126,    49.1271,  1042.7302,   107.6567,   783.4377,  -439.4431,
             5.9409,   -33.9704,  -696.5518,    57.1101,  -412.6780,  -145.5982,
           -62.0596,   312.7665,  -962.0673,   -30.8466,  -715.1412,   275.5680,
          -175.7269, -1035.7534,  -586.1561,   514.3160,  -368.1536,     5.5575,
           475.0209,   529.7679], grad_fn=<DivBackward0>))

In [67]:
sum = e[range(n), Yb].sum()
dlogits[range(n), Yb]

tensor([-0.9816, -0.9757, -0.9891, -0.9449, -0.9733, -0.9755, -0.9661, -0.9706,
        -0.9891, -0.9723, -0.9859, -0.9520, -0.9693, -0.9680, -0.9380, -0.9708,
        -0.9533, -0.9643, -0.9671, -0.9774, -0.9191, -0.9681, -0.9368, -0.9765,
        -0.9632, -0.9129, -0.9445, -0.9815, -0.9553, -0.9693, -0.9808, -0.9818],
       grad_fn=<IndexBackward0>)

### answer

In [ ]:
# YO!!

# (individual) loss = -ln(p_i)

# p_i = e^x / S(e^X)
# -ln(e^x / S(e^x)) = -[ln(e^x) - ln(S(e^x))]
# = ln(S(e^x)) - ln(e^x)

# d/dx = 1/(S(e^x)) * e^x - 1 = p_i - 1
# d/dx (unselected) = p_i

In [155]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n #shit this was the problem
dlogits[0]
# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 6.28642737865448e-09


In [1047]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dx_i = dhpreact * bngain

# y * xi + B
# xi = (x - u) / sqrt(v + e)
# dxi/dv =  d/dv (v+e)**-0.5 = -0.5 * (v + e)**-1.5
# all of the individual xi's depend on v -> sum the impact
# dL/dv = dL/dxi * dxi/dv = y_i * y * -0.5 * (v + e)**-1.5

# wtf...
dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1) * bnraw*(dhpreact * bnraw).sum(0))
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [194]:
from tqdm import tqdm

# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():
  # kick off optimization
  for i in tqdm(range(max_steps)):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    # loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    # YOUR CODE HERE :)
    dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None

    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n #shit this was the problem
    # === 2nd layer backprop ====
    # little bit freaky, but ok...
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0) # first broadcast to [32, 27], local derivates is just ones, then with global derivative it's like taking a sum
    # === tanh ===
    dhpreact = (1 - h**2) * dh
    # === batch norm ===
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1) * bnraw*(dhpreact * bnraw).sum(0))
    # === 1st layer ===
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = 1.0 * dhprebn.sum(0)
    # === embedding ===
    demb = 1.0 * dembcat.view(emb.shape) #unview back
    dC = torch.zeros_like(C)

    for i, batch in enumerate(Xb):
        for j, chari in enumerate(batch): 
            dC[chari] += 1.0 * demb[i, j] #probs not the best way...

    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())

      # if not torch.allclose(p.grad, grad):
        #  print('Different with: ', torch.sub(p.grad, grad).abs().max())
        # pass

      # print(len(grads), p.shape)
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')

    lossi.append(loss.log10().item())

    if i >= 100: # TODO: delete early breaking when you're ready to train the full net
      break

12297


100%|██████████| 200000/200000 [03:00<00:00, 1107.57it/s]


In [199]:
# useful for checking your gradients
for p,g in zip(parameters, grads):
  cmp(str(tuple(p.shape)), g, p)

TypeError: all() received an invalid combination of arguments - got (bool), but expected one of:
 * (Tensor input, *, Tensor out = None)
 * (Tensor input, tuple of ints dim = None, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, int dim, bool keepdim = False, *, Tensor out = None)
 * (Tensor input, name dim, bool keepdim = False, *, Tensor out = None)


In [197]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [176]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.1712758541107178
val 2.195324420928955


In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [198]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))

carmahzamille.
khi.
myli.
tety.
skanden.
jazon.
nadelly.
caraqui.
nellara.
chriivia.
leigh.
ham.
joce.
quinton.
lilea.
jadbi.
wavero.
dearisi.
jaxen.
druly.
